# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234132100355                   -0.50    7.0
  2   -7.250051537018       -1.80       -1.39    1.0
  3   -7.251297383934       -2.90       -2.05    4.0
  4   -7.251252104776   +   -4.34       -2.18    4.0
  5   -7.251316536083       -4.19       -2.49    3.0
  6   -7.251337360324       -4.68       -3.04    2.0
  7   -7.251338769580       -5.85       -4.00    2.0
  8   -7.251338787083       -7.76       -4.02    4.0
  9   -7.251338797638       -7.98       -4.50    3.0
 10   -7.251338798485       -9.07       -4.90    2.0
 11   -7.251338798700       -9.67       -5.93    3.0
 12   -7.251338798704      -11.45       -6.11    3.0
 13   -7.251338798705      -12.29       -6.59    2.0
 14   -7.251338798705      -13.56       -7.29    2.0
 15   -7.251338798705      -14.45       -7.64    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08474654632843664
[ Info: Arnoldi iteration step 2: normres = 0.5746213673387883
[ Info: Arnoldi iteration step 3: normres = 0.6658854336010491
[ Info: Arnoldi iteration step 4: normres = 0.2659378601982471
[ Info: Arnoldi iteration step 5: normres = 0.24946477233672626
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.92e-03, 3.77e-02, 1.51e-01, 1.93e-01, 2.87e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4854992749441081
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.44e-03, 4.05e-02, 4.68e-01, 7.77e-02, 8.69e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09647722421686772
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.36e-04, 9.41e-03, 4.42e-02, 2.81e-02, 6.85e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12069489595109255
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.13e-